<a href="https://colab.research.google.com/github/jqshang/CSC2541/blob/main/Dataload.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install MDAnalysis MDAnalysisData pandas numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 91.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 2.0 MB/s eta 0:00:00


In [2]:
import MDAnalysis as mda
from MDAnalysisData import datasets
adk = datasets.fetch_adk_transitions_DIMS()

/usr/local/lib/python3.12/dist-packages/MDAnalysisData/base.py:34: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import resource_string
DIMS.tar.gz: 731MB [02:28, 4.92MB/s]                           
/usr/local/lib/python3.12/dist-packages/MDAnalysisData/adk_transitions.py:177: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(path=data_location)


In [3]:
print(adk.DESCR)

.. -*- coding: utf-8 -*-

.. _`adk-transitions-DIMS-dataset`:

AdK DIMS transitions ensemble dataset

The macromolecular conformational transition between the closed
conformation of apo-adenylate kinase from E. coli (EcAdK) to the open
conformation was sampled with two methods: (1) dynamic importance
sampling molecular dynamics (DIMS MD) [Perilla2009]_, and (2)
Framework Rigidity Optimized Dynamics Algorithm (FRODA)
[Farrell2010]_. Each ensemble of independently generated paths
contains 200 trajectories in the CHARMM DCD format.

This data set contains the **DIMS** ensemble.

DIMS AdK (implicit solvent) with dynamic importance sampling MD from
closed (1AKE) to open (4AKE). CHARMM 22 force field. Topology file:
adk4ake.psf


Notes
-----

Data set characteristics:

 :size: 757 MB
 :number of trajectories:  200	
 :number of frames:  varies
 :number of atoms: 3341
 :creator: Sean L. Seyler
 :URL:  `https://doi.org/10.6084/m9.figshare.7165306.v2 <https://doi.org/10.6084/m9.figshare.7165306.

In [14]:
adk.N_trajectories, adk.topology

(200, '/root/MDAnalysis_data/adk_transitions_DIMS/DIMS/topologies/adk4ake.psf')

In [24]:
u = mda.Universe(adk.topology, adk.trajectories[0])

/usr/local/lib/python3.12/dist-packages/MDAnalysis/coordinates/DCD.py:171: DeprecationWarning: DCDReader currently makes independent timesteps by copying self.ts while other readers update self.ts inplace. This behavior will be changed in 3.0 to be the same as other readers. Read more at https://github.com/MDAnalysis/mdanalysis/issues/3889 to learn if this change in behavior might affect you.
  warnings.warn("DCDReader currently makes independent timesteps"


In [45]:
u.trajectory[0]

< Timestep 0 >

In [25]:
# Select all residues that have a Cα atom (standard proteins)
ca = u.select_atoms("protein and name CA")
n_res = ca.n_atoms
n_frames = len(u.trajectory)

In [26]:
n_res, n_frames

(214, 96)

In [27]:
u.trajectory[0]

< Timestep 0 >

In [28]:
import numpy as np
import pandas as pd

In [29]:
ref = ca.positions.copy()

disp = np.empty((n_frames, n_res), dtype=np.float32)

for i in range(len(u.trajectory)):
    delta = ca.positions - ref
    disp[i, :] = np.linalg.norm(delta, axis=1)

disp

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [30]:
res_ids = [f"{r.resname}{r.resid}" for r in ca.residues]
df = pd.DataFrame(disp, columns=res_ids)
df.index.name = "frame"

In [31]:
df

,MET1,ARG2,ILE3,ILE4,LEU5,LEU6,GLY7,ALA8,PRO9,GLY10,...,VAL205,ARG206,ALA207,ASP208,LEU209,GLU210,LYS211,ILE212,LEU213,GLY214
frame,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
92,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
93,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
